**Задание** 

Найти три машины, получившие наибольшее количество чаевых в апреле 2018 года, и выведите, как изменялась эта сумма в процентах в последующие месяцы по сравнению с предыдущим в таблице со столбцами taxi_id, year_month, tips_sum, tips_change. В качестве даты поездки возьмите дату начала поездки. В качестве источника данных используйте bigquery-public-data.chicago_taxi_trips. 

**Просмотр и анализ базы данных** 

In [ ]:
SELECT *
FROM bigquery-public-data.chicago_taxi_trips.taxi_trips;  

**Решение задания**

In [ ]:
WITH top_tipping_taxis AS (
SELECT
    taxi_id,
    SUM(tips) AS tips_sum
  FROM
    bigquery-public-data.chicago_taxi_trips.taxi_trips
  WHERE
    EXTRACT(MONTH FROM trip_start_timestamp) = 4
    AND EXTRACT(YEAR FROM trip_start_timestamp) = 2018
  GROUP BY
    taxi_id
  ORDER BY
    tips_sum DESC
  LIMIT
    3),
tips_by_month AS
(SELECT
    taxi_id,
    FORMAT_TIMESTAMP('%Y-%m', trip_start_timestamp) AS year_month,
    SUM(tips) AS tips_sum
  FROM
    bigquery-public-data.chicago_taxi_trips.taxi_trips
  WHERE
    taxi_id IN (SELECT taxi_id FROM top_tipping_taxis)
    AND FORMAT_TIMESTAMP('%Y-%m', trip_start_timestamp) >= '2018-04'
  GROUP BY
    taxi_id, year_month
  ORDER BY year_month)
  SELECT taxi_id,
  year_month,
  tips_sum,
  ROUND(((tips_sum - LAG(tips_sum, 1) OVER (PARTITION BY taxi_id ORDER BY year_month)) / LAG(tips_sum, 1) OVER (PARTITION BY taxi_id ORDER BY year_month)) * 100, 2) AS tips_change
FROM
  tips_by_month
ORDER BY
  taxi_id, year_month;

**Ссылка на выгрузку таблицы с результатом в google sheets  - ** 
https://docs.google.com/spreadsheets/d/1bQLgQyjbCLFUYI2a8GvyXA_Pf9fnSksWB7AuQ49KP8k/edit?usp=sharing 